In [ ]:
import pandas as pd
import os
    
datasets = {"Retail_Rocket": {"path": '/home/jy1559/Mar2025_Module/Datasets/Retail_Rocket',
                              "file_names": ['category_tree.csv', 'events.csv', 'item_properties_part1.csv', 'item_properties_part2.csv']},
            "Diginetica": {"path": '/home/jy1559/Mar2025_Module/Datasets/Diginetica',
                              "file_names": ['product-categories.csv', 'products.csv', 'train-clicks.csv', 'train-item-views.csv', 'train-purchases.csv', 'train-queries.csv']},
            "LFM-BeyMS": {"path": '/home/jy1559/Mar2025_Module/Datasets/LFM-BeyMS/dataset',
                              "file_names": ['beyms.csv', 'events.csv', 'genre_annotations.csv', 'mainstreaminess.csv', 'ms.csv', 'user_groups.csv']},
            "Beauty": {"path": '/home/jy1559/Mar2025_Module/Datasets/Amazon',
                "file_names": ['All_Beauty.jsonl', 'meta_All_Beauty.jsonl']},
            "Game": {"path": '/home/jy1559/Mar2025_Module/Datasets/Amazon',
                "file_names": ['Video_Games.jsonl', 'meta_Video_Games.jsonl']}}
dataset = datasets["LFM-BeyMS"]
directory_path = dataset['path']
file_names = dataset["file_names"]
for name in file_names:
    file_path = os.path.join(directory_path, name)
    try:
        if 'csv' in file_path:
            df = pd.read_csv(file_path)
        elif 'json' in file_path:
            df = pd.read_json(file_path, lines=True)
        print(f"First 5 rows of {name}:")
        print(df.head(), "\n")
    except FileNotFoundError:
        print(f"File {name} not found in the directory {directory_path}.")
    except pd.errors.EmptyDataError:
        print(f"File {name} is empty.")
    except pd.errors.ParserError:
        print(f"Error parsing {name}. Please check the file for inconsistencies.")

First 5 rows of beyms.csv:
   user_id
0  1049656
1  1055118
2  1056935
3  1070023
4  1072752 

First 5 rows of events.csv:
    user_id  artist_id  album_id  track_id   timestamp
0  31435741         21        31        53  1370977938
1  31435741         21        31        53  1370977728
2  31435741         21        31        53  1370977518
3  31435741         21        31        53  1370977308
4  31435741         21        31        53  1370977098 

First 5 rows of genre_annotations.csv:
   Unnamed: 0  track_id                                             genres
0           1      4868  ['soul', 'pop', 'singersongwriter', 'blues', '...
1           2      2900  ['electronic', 'indiepop', 'shoegaze', 'dreamp...
2           5    572665  ['soul', 'pop', 'singersongwriter', 'blues', '...
3           6      2897  ['indierock', 'electronic', 'indiepop', 'postp...
4           7     15100  ['folk', 'indiefolk', 'banjo', 'folkrock', 'bl... 

First 5 rows of mainstreaminess.csv:
   Unnamed: 0   u

In [ ]:
import ast

def safe_eval(val, expected_type):
    """
    문자열 형태의 리스트나 딕셔너리를 안전하게 평가하고, 
    만약 평가에 실패하거나 타입이 맞지 않으면 빈 값을 반환합니다.
    """
    try:
        evaluated = ast.literal_eval(val) if isinstance(val, str) else val
        if isinstance(evaluated, expected_type):
            return evaluated
    except Exception:
        pass
    return None

def construct_item_sentence(row):
    parts = []
    
    # Title
    title = row.get('title', '').strip()
    if title:
        parts.append(f"Title: {title}.")
    
    # Rating 정보
    avg_rating = row.get('average_rating')
    rating_number = row.get('rating_number')
    if avg_rating is not None and rating_number is not None:
        parts.append(f"Average rating: {avg_rating}, based on {rating_number} reviews.")
    
    # Features (리스트 형태)
    features = safe_eval(row.get('features', ''), list)
    if features:
        # 리스트가 비어있지 않으면 각 항목을 문자열로 연결
        features_text = ", ".join(str(f).strip() for f in features if f)
        if features_text:
            parts.append(f"Features: {features_text}.")
    
    # Description (리스트 형태)
    description = safe_eval(row.get('description', ''), list)
    if description:
        description_text = " ".join(str(d).strip() for d in description if d)
        if description_text:
            parts.append(f"Description: {description_text}.")
    
    # Store (문자열)
    store = row.get('store', '')
    if store:
        parts.append(f"Store: {store.strip()}.")
    
    # Details (딕셔너리 형태)
    details = safe_eval(row.get('details', ''), dict)
    if details:
        # key: value 형태로 변환
        details_text = ", ".join(f"{k}: {v}" for k, v in details.items() if v)
        if details_text:
            parts.append(f"Details: {details_text}.")
    
    # 모든 부분을 하나의 문장으로 합치기
    return " ".join(parts)

# 예시: df는 amazon metadata가 담긴 DataFrame
# parent_asin을 key로 하는 dictionary 생성
file_path = os.path.join(directory_path, dataset["file_names"][2])
item_sentences = {}
for idx, row in df.iterrows():
    asin = row.get('parent_asin', None)
    if asin:
        sentence = construct_item_sentence(row)
        item_sentences[asin] = sentence

# 결과 확인: 특정 아이템의 문장을 출력
example_asin = list(item_sentences.keys())[1]
print(f"ASIN: {example_asin}\nSentence: {item_sentences[example_asin]}")

ASIN: B076WQZGPM
Sentence: Title: Yes to Tomatoes Detoxifying Charcoal Cleanser (Pack of 2) with Charcoal Powder, Tomato Fruit Extract, and Gingko Biloba Leaf Extract, 5 fl. oz.. Average rating: 4.5, based on 3 reviews. Store: Yes To. Details: Item Form: Powder, Skin Type: Acne Prone, Brand: Yes To, Age Range (Description): Adult, Unit Count: 10 Fl Oz, Is Discontinued By Manufacturer: No, Item model number: SG_B076WQZGPM_US, UPC: 653801351125, Manufacturer: Yes to Tomatoes.


In [ ]:
from tqdm.auto import tqdm
file_path = os.path.join(directory_path, dataset["file_names"][0])
df = pd.read_json(file_path, lines=True)
# 1. 타임스탬프를 자연어 형태로 변환 (연-월-일 시:분:초)
df['datetime'] = pd.to_datetime(df['timestamp'], unit='ms')
df['datetime_str'] = df['datetime'].dt.strftime('%Y-%m-%d %H:%M:%S')
# 날짜만 추출해서 그룹핑용으로 사용 (연-월-일)
df['date'] = df['datetime'].dt.date

# 2. 필요한 컬럼만 선택하여 리뷰 정보를 딕셔너리 형태로 정리
def get_review_info(row):
    return {
        "parent_asin": row['parent_asin'],
        "asin": row['asin'],
        "timestamp": row['datetime_str'],
        "review_info": {
        "rating": row['rating'],
        "title": row['title'],
        "text": row['text'],
        "helpful_vote": row['helpful_vote']}
    }

df['review_info'] = df.apply(get_review_info, axis=1)

# 3. user_id별로 그룹화한 뒤, 같은 날짜별로 리뷰 그룹을 생성
# 결과: { user_id: [ [review1, review2, ...] (같은 날), [review3, ...], ... ] }
user_groups = {}
for user, group in tqdm(df.groupby('user_id')):
    # 날짜별 그룹 (리스트 순서는 날짜 오름차순)
    day_groups = []
    for date, day_df in group.groupby('date'):
        # 해당 날짜에 해당하는 리뷰 정보 리스트
        reviews = day_df['review_info'].tolist()
        day_groups.append(reviews)
    user_groups[user] = day_groups

# 만약 user_id를 제거하고, 모든 사용자의 데이터만 리스트로 저장하려면:
all_users_data = list(user_groups.values())
import pandas as pd

# 파일 경로 (각 경로는 실제 경로에 맞게 수정)
events_path = '/path/to/events.csv'
genres_path = '/path/to/genre_annotations.csv'
beyms_path = '/path/to/beyms.csv'

# CSV 파일 로드
df_events = pd.read_csv(events_path)
df_genres = pd.read_csv(genres_path)
df_beyms = pd.read_csv(beyms_path)

# 1. 타임스탬프 변환
# events.csv의 timestamp가 초 단위라고 가정 (예: 1370977938)
df_events['datetime'] = pd.to_datetime(df_events['timestamp'], unit='s')
df_events['datetime_str'] = df_events['datetime'].dt.strftime('%Y-%m-%d %H:%M:%S')
df_events['date'] = df_events['datetime'].dt.date  # 그룹화용 날짜 정보

# 2. genre_annotations와 병합 (track_id 기준)
# genre_annotations.csv에서 필요한 컬럼만 사용 (여기서는 'track_id'와 'genres')
df_events = df_events.merge(df_genres[['track_id', 'genres']], on='track_id', how='left')

# 3. beyms.csv에 있는 사용자만 필터링 (관심 사용자)
beyms_user_ids = set(df_beyms['user_id'].unique())
df_events = df_events[df_events['user_id'].isin(beyms_user_ids)]

# 4. user_id별, 그리고 같은 날짜(연-월-일)별로 그룹화하여 세션 구성
user_sessions = {}import pandas as pd

# 파일 경로 (각 경로는 실제 경로에 맞게 수정)
events_path = '/path/to/events.csv'
genres_path = '/path/to/genre_annotations.csv'
beyms_path = '/path/to/beyms.csv'

# CSV 파일 로드
df_events = pd.read_csv(events_path)
df_genres = pd.read_csv(genres_path)
df_beyms = pd.read_csv(beyms_path)

# 1. 타임스탬프 변환
# events.csv의 timestamp가 초 단위라고 가정 (예: 1370977938)
df_events['datetime'] = pd.to_datetime(df_events['timestamp'], unit='s')
df_events['datetime_str'] = df_events['datetime'].dt.strftime('%Y-%m-%d %H:%M:%S')
df_events['date'] = df_events['datetime'].dt.date  # 그룹화용 날짜 정보

# 2. genre_annotations와 병합 (track_id 기준)
# genre_annotations.csv에서 필요한 컬럼만 사용 (여기서는 'track_id'와 'genres')
df_events = df_events.merge(df_genres[['track_id', 'genres']], on='track_id', how='left')

# 3. beyms.csv에 있는 사용자만 필터링 (관심 사용자)
beyms_user_ids = set(df_beyms['user_id'].unique())
df_events = df_events[df_events['user_id'].isin(beyms_user_ids)]

# 4. user_id별, 그리고 같은 날짜(연-월-일)별로 그룹화하여 세션 구성
user_sessions = {}
for user_id, user_group in df_events.groupby('user_id'):
    sessions = []
    for date, day_group in user_group.groupby('date'):
        events_list = []
        for idx, row in day_group.iterrows():
            event = {
                "artist_id": row["artist_id"],
                "album_id": row["album_id"],
                "track_id": row["track_id"],
                "datetime": row["datetime_str"],
                "genres": row["genres"] if pd.notna(row["genres"]) else ""
            }
            events_list.append(event)
        sessions.append(events_list)
    user_sessions[user_id] = sessions

# 예시: 첫 번째 사용자에 대한 세션 출력 (user_id는 key로 남아있음)
example_user = list(user_sessions.keys())[0]
print(f"User {example_user} sessions:")
for i, session in enumerate(user_sessions[example_user], 1):
    print(f"  Session {i}:")
    for event in session:
        print("    ", event)
        
# 만약 user_id를 제거하고 단순히 모든 사용자의 세션 리스트가 필요하다면:
all_user_sessions = list(user_sessions.values())

for user_id, user_group in df_events.groupby('user_id'):
    sessions = []
    for date, day_group in user_group.groupby('date'):
        events_list = []
        for idx, row in day_group.iterrows():
            event = {
                "artist_id": row["artist_id"],
                "album_id": row["album_id"],
                "track_id": row["track_id"],
                "datetime": row["datetime_str"],
                "genres": row["genres"] if pd.notnaimport pandas as pd

# 파일 경로 (각 경로는 실제 경로에 맞게 수정)
events_path = '/path/to/events.csv'
genres_path = '/path/to/genre_annotations.csv'
beyms_path = '/path/to/beyms.csv'

# CSV 파일 로드
df_events = pd.read_csv(events_path)
df_genres = pd.read_csv(genres_path)
df_beyms = pd.read_csv(beyms_path)

# 1. 타임스탬프 변환
# events.csv의 timestamp가 초 단위라고 가정 (예: 1370977938)
df_events['datetime'] = pd.to_datetime(df_events['timestamp'], unit='s')
df_events['datetime_str'] = df_events['datetime'].dt.strftime('%Y-%m-%d %H:%M:%S')
df_events['date'] = df_events['datetime'].dt.date  # 그룹화용 날짜 정보

# 2. genre_annotations와 병합 (track_id 기준)
# genre_annotations.csv에서 필요한 컬럼만 사용 (여기서는 'track_id'와 'genres')
df_events = df_events.merge(df_genres[['track_id', 'genres']], on='track_id', how='left')

# 3. beyms.csv에 있는 사용자만 필터링 (관심 사용자)
beyms_user_ids = set(df_beyms['user_id'].unique())
df_events = df_events[df_events['user_id'].isin(beyms_user_ids)]

# 4. user_id별, 그리고 같은 날짜(연-월-일)별로 그룹화하여 세션 구성
user_sessions = {}
for user_id, user_group in df_events.groupby('user_id'):
    sessions = []
    for date, day_group in user_group.groupby('date'):
        events_list = []
        for idx, row in day_group.iterrows():
            event = {
                "artist_id": row["artist_id"],
                "album_id": row["album_id"],
                "track_id": row["track_id"],
                "datetime": row["datetime_str"],
                "genres": row["genres"] if pd.notna(row["genres"]) else ""
            }
            events_list.append(event)
        sessions.append(events_list)
    user_sessions[user_id] = sessions

# 예시: 첫 번째 사용자에 대한 세션 출력 (user_id는 key로 남아있음)
example_user = list(user_sessions.keys())[0]
print(f"User {example_user} sessions:")
for i, session in enumerate(user_sessions[example_user], 1):
    print(f"  Session {i}:")
    for event in session:
        print("    ", event)
        
# 만약 user_id를 제거하고 단순히 모든 사용자의 세션 리스트가 필요하다면:
all_user_sessions = list(user_sessions.values())
(row["genres"]) else ""
            }
            events_list.append(event)
        sessions.append(events_list)
    user_sessions[user_id] = sessions

# 예시: 첫 번째 사용자에 대한 세션 출력 (user_id는 key로 남아있음)
example_user = list(user_sessions.keys())[0]
print(f"User {example_user} sessions:")
for i, session in enumerate(user_sessions[example_user], 1):
    print(f"  Session {i}:")
    for event in session:
        print("    ", event)
        
# 만약 user_id를 제거하고 단순히 모든 사용자의 세션 리스트가 필요하다면:
all_user_sessions = list(user_sessions.values())

# 결과 예시 출력 (첫 번째 user의 데이터를 출력)
example_user = list(user_groups.keys())[0]
print(f"User {example_user} grouped reviews:")
for i, day_group in enumerate(user_groups[example_user], 2):
    print(f"  Day group {i}:")
    for review in day_group:
        print("   ", review)

/home/jy1559/.conda/envs/first/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
 17%|█▋        | 105142/631986 [00:36<02:48, 3131.40it/s]

In [ ]:
example_user = list(user_groups.keys())[0]
print(user_groups[example_user])

NameError: name 'user_groups' is not defined